# ImageToImage Translation

In [6]:
# Define preprocessing function to resize images
def preprocess_image(image, target_size=(64, 64)):
    image = tf.image.resize(image, target_size)
    return image


In [7]:
# Preprocess the images (real_faces and real_sketches)
def preprocess_train_data(real_faces, real_sketches):
    real_faces = preprocess_image(real_faces)
    real_sketches = preprocess_image(real_sketches)
    return real_faces, real_sketches


In [26]:
import os
from tensorflow.keras.metrics import Mean
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa  # For Instance Normalization
import matplotlib.pyplot as plt

# Hyperparameters
learning_rate = 2e-4
beta1 = 0.5
beta2 = 0.999
batch_size = 32
EPOCHS = 100
image_shape = (64, 64, 3)  # Shape of the generated face images
sketch_shape = (64, 64, 1)  # Shape of the generated sketches
output_dir = "cycleGAN_output"  # Directory to save generated images
model_dir = "saved_models"  # Directory to save the model

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, "fake_images"), exist_ok=True)  # Directory for fake images
os.makedirs(os.path.join(output_dir, "fake_sketches"), exist_ok=True)  # Directory for fake sketches

# Loss functions
pixel_loss = tf.keras.losses.MeanAbsoluteError()
cycle_loss = tf.keras.losses.MeanAbsoluteError()
contextual_loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

# Instance Normalization layer
def instance_norm():
    return tfa.layers.InstanceNormalization(axis=-1)

# Define the generator (for both sketch-to-face and face-to-sketch)
def build_generator(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # Downsampling
    x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)  # 64x64 -> 32x32
    x = instance_norm()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(x)  # 32x32 -> 16x16
    x = instance_norm()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(256, kernel_size=4, strides=2, padding='same')(x)  # 16x16 -> 8x8
    x = instance_norm()(x)
    x = layers.LeakyReLU()(x)

    # Upsampling
    x = layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same')(x)  # 8x8 -> 16x16
    x = instance_norm()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same')(x)  # 16x16 -> 32x32
    x = instance_norm()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding='same')(x)  # 32x32 -> 64x64
    x = instance_norm()(x)
    x = layers.LeakyReLU()(x)

    # Output 1 channel for sketches, 3 channels for faces
    output_channels = 1 if input_shape[-1] == 3 else 3  # Expect sketch if input is face, and vice versa
    x = layers.Conv2D(output_channels, kernel_size=7, padding='same', activation='tanh')(x)

    return tf.keras.Model(inputs, x)

# Define the discriminator
def build_discriminator(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
    x = instance_norm()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(256, kernel_size=4, strides=2, padding='same')(x)
    x = instance_norm()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    return tf.keras.Model(inputs, x)

# Instantiate models
G1 = build_generator(image_shape)  # Face to sketch
G2 = build_generator(sketch_shape)  # Sketch to face
D1 = build_discriminator(sketch_shape)  # Discriminator for sketches
D2 = build_discriminator(image_shape)  # Discriminator for real faces


generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Build the optimizers with the model variables
generator_optimizer.build(G1.trainable_variables + G2.trainable_variables)
discriminator_optimizer.build(D1.trainable_variables + D2.trainable_variables)


# CycleGAN loss functions
def generator_loss(fake_output):
    return contextual_loss(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = contextual_loss(tf.ones_like(real_output), real_output)
    fake_loss = contextual_loss(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def cycle_consistency_loss(real_image, cycled_image):
    return cycle_loss(real_image, cycled_image)

# Training step for CycleGAN
@tf.function
def train_step(real_faces, real_sketches):
    with tf.GradientTape(persistent=True) as tape:
        # Generate sketches from faces and vice versa
        fake_sketches = G1(real_faces, training=True)
        fake_faces = G2(real_sketches, training=True)

        # Reconstruct images
        cycled_faces = G2(fake_sketches, training=True)
        cycled_sketches = G1(fake_faces, training=True)

        # Discriminator outputs
        real_sketch_output = D1(real_sketches, training=True)
        fake_sketch_output = D1(fake_sketches, training=True)

        real_face_output = D2(real_faces, training=True)
        fake_face_output = D2(fake_faces, training=True)

        # Generator losses
        G1_loss = generator_loss(fake_sketch_output) + cycle_consistency_loss(real_faces, cycled_faces)
        G2_loss = generator_loss(fake_face_output) + cycle_consistency_loss(real_sketches, cycled_sketches)

        # Discriminator losses
        D1_loss = discriminator_loss(real_sketch_output, fake_sketch_output)
        D2_loss = discriminator_loss(real_face_output, fake_face_output)

    # Get gradients and apply them
    G1_gradients = tape.gradient(G1_loss, G1.trainable_variables)
    G2_gradients = tape.gradient(G2_loss, G2.trainable_variables)
    D1_gradients = tape.gradient(D1_loss, D1.trainable_variables)
    D2_gradients = tape.gradient(D2_loss, D2.trainable_variables)

    generator_optimizer.apply_gradients(zip(G1_gradients, G1.trainable_variables))
    generator_optimizer.apply_gradients(zip(G2_gradients, G2.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(D1_gradients, D1.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(D2_gradients, D2.trainable_variables))

    return G1_loss, G2_loss, D1_loss, D2_loss  # Return the losses

# Prepare dataset
def load_data(sketch_dir, image_dir):
    sketch_images = []
    real_images = []
    
    for sketch_file in os.listdir(sketch_dir):
        if sketch_file.endswith(".png") or sketch_file.endswith(".jpg"):
            sketch_path = os.path.join(sketch_dir, sketch_file)
            image_path = os.path.join(image_dir, sketch_file)  # Assuming matching filenames
            
            # Load and preprocess sketch (64x64 grayscale)
            sketch = tf.keras.preprocessing.image.load_img(sketch_path, color_mode='grayscale', target_size=(64, 64))
            sketch = tf.keras.preprocessing.image.img_to_array(sketch) / 255.0
            sketch_images.append(sketch)

            # Load and preprocess real image (64x64 RGB)
            real_image = tf.keras.preprocessing.image.load_img(image_path, target_size=(64, 64))  # Resizing to 64x64
            real_image = tf.keras.preprocessing.image.img_to_array(real_image) / 255.0
            real_images.append(real_image)

    # Convert to NumPy arrays
    return np.array(sketch_images), np.array(real_images)

sketches_folder = '/Users/zainabaslam/Local Docs/GenAI-A2/Dataset/train/sketches'  # Directory containing sketches
images_folder = '/Users/zainabaslam/Local Docs/GenAI-A2/Dataset/train/photos'     
train_sketches, train_images = load_data(sketches_folder, images_folder)

train_dataset = tf.data.Dataset.from_tensor_slices((train_sketches, train_images))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    
    # Reset the loss metrics
    G1_loss_avg = Mean()
    G2_loss_avg = Mean()
    D1_loss_avg = Mean()
    D2_loss_avg = Mean()

    for step, (real_sketches, real_faces) in enumerate(train_dataset):
        if step >= 32:  # Only run for 32 batches
            break
        
        G1_loss, G2_loss, D1_loss, D2_loss = train_step(real_faces, real_sketches)

        # Update average losses
        G1_loss_avg.update_state(G1_loss)
        G2_loss_avg.update_state(G2_loss)
        D1_loss_avg.update_state(D1_loss)
        D2_loss_avg.update_state(D2_loss)

        # Print the losses for the current batch
        print(f'Batch {step + 1}, G1 Loss: {G1_loss.numpy():.4f}, G2 Loss: {G2_loss.numpy():.4f}, D1 Loss: {D1_loss.numpy():.4f}, D2 Loss: {D2_loss.numpy():.4f}')

    # Log epoch results
    print(f'Epoch {epoch + 1} Average -> G1 Loss: {G1_loss_avg.result()}, G2 Loss: {G2_loss_avg.result()}, D1 Loss: {D1_loss_avg.result()}, D2 Loss: {D2_loss_avg.result()}')
    
    # Save fake images and sketches at the end of each epoch
    fake_images = G2(real_sketches)  # Generate fake faces from real sketches
    fake_sketches = G1(real_faces)  # Generate fake sketches from real faces

    # Ensure to use the correct number of images to save
    for i in range(min(batch_size, fake_images.shape[0])):  # Adjust to the actual number of generated images
        # Save generated fake faces
        img_array = (fake_images[i].numpy() * 255).astype(np.uint8)  # Convert tensor to numpy array
        tf.keras.preprocessing.image.save_img(os.path.join(output_dir, "fake_images", f"fake_face_epoch_{epoch + 1}_img_{i + 1}.png"), img_array)

        # Save generated fake sketches
        sketch_array = (fake_sketches[i].numpy() * 255).astype(np.uint8)  # Convert tensor to numpy array
        tf.keras.preprocessing.image.save_img(os.path.join(output_dir, "fake_sketches", f"fake_sketch_epoch_{epoch + 1}_img_{i + 1}.png"), sketch_array)

print("Training completed.")


Epoch 1/100
Batch 1, G1 Loss: 1.3369, G2 Loss: 2.2836, D1 Loss: 1.6694, D2 Loss: 1.4059
Batch 2, G1 Loss: 0.7684, G2 Loss: 1.3253, D1 Loss: 2.5402, D2 Loss: 2.6238
Batch 3, G1 Loss: 0.9999, G2 Loss: 1.3330, D1 Loss: 1.7343, D2 Loss: 2.3830
Batch 4, G1 Loss: 1.6713, G2 Loss: 2.2116, D1 Loss: 0.6262, D2 Loss: 0.6089
Batch 5, G1 Loss: 1.2475, G2 Loss: 1.4250, D1 Loss: 1.0459, D2 Loss: 1.8272
Batch 6, G1 Loss: 1.4370, G2 Loss: 1.7080, D1 Loss: 0.9162, D2 Loss: 1.2866
Batch 7, G1 Loss: 1.1752, G2 Loss: 1.9318, D1 Loss: 1.1045, D2 Loss: 0.8458
Batch 8, G1 Loss: 1.5356, G2 Loss: 1.4222, D1 Loss: 0.6887, D2 Loss: 1.3773
Batch 9, G1 Loss: 1.4618, G2 Loss: 2.3071, D1 Loss: 0.8114, D2 Loss: 0.6073
Batch 10, G1 Loss: 2.1694, G2 Loss: 1.1219, D1 Loss: 0.5640, D2 Loss: 2.1010
Batch 11, G1 Loss: 1.7287, G2 Loss: 3.6728, D1 Loss: 0.8271, D2 Loss: 0.2397
Batch 12, G1 Loss: 2.4116, G2 Loss: 0.8159, D1 Loss: 0.3850, D2 Loss: 3.0139
Batch 13, G1 Loss: 1.9183, G2 Loss: 5.4205, D1 Loss: 0.6376, D2 Loss: 0.2

/var/folders/5_/6fblyny50t1ffbsx4k05bgw80000gn/T/ipykernel_47728/3570531442.py:221: RuntimeWarning: invalid value encountered in cast
  img_array = (fake_images[i].numpy() * 255).astype(np.uint8)  # Convert tensor to numpy array
/var/folders/5_/6fblyny50t1ffbsx4k05bgw80000gn/T/ipykernel_47728/3570531442.py:225: RuntimeWarning: invalid value encountered in cast
  sketch_array = (fake_sketches[i].numpy() * 255).astype(np.uint8)  # Convert tensor to numpy array


Epoch 2/100
Batch 1, G1 Loss: 0.7539, G2 Loss: 8.2144, D1 Loss: 1.7634, D2 Loss: 0.3604
Batch 2, G1 Loss: 3.0385, G2 Loss: 6.4097, D1 Loss: 1.0710, D2 Loss: 0.1661
Batch 3, G1 Loss: 1.6272, G2 Loss: 0.7198, D1 Loss: 0.7252, D2 Loss: 1.2460
Batch 4, G1 Loss: 1.6435, G2 Loss: 4.5464, D1 Loss: 0.8195, D2 Loss: 0.6102
Batch 5, G1 Loss: 2.1699, G2 Loss: 1.0066, D1 Loss: 0.6628, D2 Loss: 1.1074
Batch 6, G1 Loss: 1.8035, G2 Loss: 4.4633, D1 Loss: 1.0076, D2 Loss: 0.5811
Batch 7, G1 Loss: 1.4849, G2 Loss: 0.9140, D1 Loss: 0.9777, D2 Loss: 0.8813
Batch 8, G1 Loss: 1.7964, G2 Loss: 4.5828, D1 Loss: 0.9211, D2 Loss: 0.6683
Batch 9, G1 Loss: 1.1096, G2 Loss: 0.8857, D1 Loss: 1.2968, D2 Loss: 1.1513
Batch 10, G1 Loss: 3.2426, G2 Loss: 5.3692, D1 Loss: 1.1167, D2 Loss: 0.3936
Batch 11, G1 Loss: 0.7215, G2 Loss: 1.7785, D1 Loss: 1.9430, D2 Loss: 0.3303
Batch 12, G1 Loss: 3.8969, G2 Loss: 1.4165, D1 Loss: 1.3384, D2 Loss: 1.0684
Batch 13, G1 Loss: 0.6353, G2 Loss: 3.3823, D1 Loss: 2.3963, D2 Loss: 0.3

In [27]:
# Assuming G1 and G2 are your generator models and D1 and D2 are your discriminator models

# Save the generator and discriminator models
G1.save('./saved_model/G1_model')
G2.save('./saved_model/G2_model')
D1.save('./saved_model/D1_model')
D2.save('./saved_model/D2_model')

print("Models saved successfully.")


INFO:tensorflow:Assets written to: ./saved_model/G1_model/assets


INFO:tensorflow:Assets written to: ./saved_model/G1_model/assets


INFO:tensorflow:Assets written to: ./saved_model/G2_model/assets


INFO:tensorflow:Assets written to: ./saved_model/G2_model/assets


INFO:tensorflow:Assets written to: ./saved_model/D1_model/assets


INFO:tensorflow:Assets written to: ./saved_model/D1_model/assets


INFO:tensorflow:Assets written to: ./saved_model/D2_model/assets


INFO:tensorflow:Assets written to: ./saved_model/D2_model/assets


Models saved successfully.


In [28]:
# Assuming G1 and G2 are your generator models and D1 and D2 are your discriminator models

# Save the generator and discriminator models in HDF5 format
G1.save('./saved_model/G1_model.h5')
G2.save('./saved_model/G2_model.h5')
D1.save('./saved_model/D1_model.h5')
D2.save('./saved_model/D2_model.h5')

print("Models saved successfully.")


/opt/anaconda3/envs/panda/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Models saved successfully.
